here I have coppied lots of my code from my random forest notebook as I wanted to try XGBoosting on the same data (I should have organzied this better) adn this is why the first coding block is so large as it is doing the data pre-processing and extracition that is necessary. 

In [ ]:
import sys
!conda init
!conda activate protein_stability

import baseline as bl
import numpy as np

from PyBioMed import Pyprotein
from PyBioMed.PyProtein import CTD

train = bl.load_train_data("data/train.csv", val_split=False)
test = bl.load_test_data("data/test.csv")

train_seq, test_seq = train.sequence, test.sequence     #extract the sequences
print("test sequence total",len(train_seq))
print("train sequence total",len(test_seq))

def get_features(seq_data):
    features = [CTD.CalculateC(protein) for protein in seq_data]
    filtered_features = [{key: value for key, value in protein_features.items() if key.endswith('C1')} for protein_features in features]

    return filtered_features        #I am only testing C1 for now

train_features = get_features(train_seq)
test_features = get_features(test_seq)
print("test",len(train_features))
print("train", len(test_features))

In [3]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

In [ ]:
train_df = pd.DataFrame(train_features)
test_df = pd.DataFrame(test_features)
train_y = np.array(train.target.values.tolist())
train_y_array = train_y

In [ ]:
xgb_regressor = xgb.XGBRegressor()

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 5, 10],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.4, 0.7, 1.0],
    'colsample_bytree': [0.4, 0.7, 1.0],
}

grid_search = GridSearchCV(estimator=xgb_regressor, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
grid_search.fit(train_df, train_y_array)

best_params = grid_search.best_params_

In [4]:
best_xgb_regressor = xgb.XGBRegressor(**best_params)
best_xgb_regressor.fit(train_df, train_y_array)

predictions = best_xgb_regressor.predict(test_df)

In [5]:
test_id = test.id.values.tolist()

with open("xgboost.csv", "w") as f:
    f.write("id,target\n")
    for id, y in zip(test_id, predictions):
        f.write(f"{id},{y}\n")